We'll try to fetch the artists and tracks I listened to in a specific period from lastfm and see if we can encirh this info with spotify data (because it only allows to 50 last tracks in a beta api)
After that we can try som simple ML models on it :D 

In [1]:
import time
import json
import datetime
from math import ceil
from datetime import date, timedelta, datetime

import numpy as np
import pandas as pd
import requests_cache
import requests as re

### Pre-work

In [186]:
#Get the csv from the last request
#COLUMNS 0 -> BAND NAME
#COLUMNS 1 -> SONG NAME
#COLUMNS 2 -> PLAYED AT UNIX TIMESTAMP
tracks = pd.read_csv('data/played_tracks.csv')

In [187]:

tracks.drop(tracks.columns[2], axis = 1, inplace = True)
#clear duplicate columns
tracks = tracks.drop_duplicates()
#rename columns
tracks.rename(mapper= {'0': 'band_name', '1': 'song_name'}, axis = 1, inplace = True)

## Spotify API

### Get Song IDs

In [188]:
#create a request cache
requests_cache.install_cache('track_features_cache')

In [190]:
#define a function for the api request that returns the json response
def search_song_request(_song_name, _band_name):
    r = re.get('https://api.spotify.com/v1/search?' + 'q=artist:' + _band_name + '%20track:' + _song_name + '&market:from_token' + '&type=track&limit=50&include_external=audio' , headers = headers)
    try:
        return r.json()['tracks']['items']
    except:
        return None

#define a function to make the api request, iterate on the response and get the song id based on the artist
def get_song_id(_song_name, _band_name):
    _response = search_song_request(_song_name, _band_name)
    if _response is not None:
        for i in range(len(_response)):
            #there must be a better way of doing this byt it'll work for now: we iterate through the entire artist list - there is a lot of useless data in the response
            if _response[i]['album']['artists'][0]['name'] == _band_name:
                return _response[i]['id']
            else:
                pass
    #if no returns, we could not find id, so it returns not_found
    return 'not_found'


In [2]:
#Authentication - getting an access token
auth_response = re.post('https://accounts.spotify.com/api/token', {
    'grant_type': 'client_credentials',
    'client_id': '6cb782c1b0404843b3e5a06e8361cb6e',
    'client_secret': 'a32c28deab714fa89091395089cbca90',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#create a header with the access token
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [192]:
print("started api calls at", datetime.now())
tracks['sp_id'] = tracks.apply(lambda x: get_song_id(x['song_name'], x['band_name']), axis = 1)
print("done at", datetime.now())

started api calls at 2020-12-13 12:46:17.315754
done at 2020-12-13 12:46:30.674313


In [195]:
tracks['no_id'] = tracks['sp_id'].apply(lambda x: x == 'not_found')
tracks.groupby('no_id').count()['sp_id']

no_id
False    5081
True     1054
Name: sp_id, dtype: int64

In [197]:
tracks.to_csv('data/tracks_ids.csv', index=False)

### Get Song Features

In [4]:
#tracks = pd.read_csv('data_extractions/tracks_ids.csv')
#we can only get data for tracks with a Spotify ID
tracks = tracks[tracks['no_id']==False]

In [5]:
def get_song_features(_ids):
    r = re.get('https://api.spotify.com/v1/audio-features?ids=' + _ids, headers=headers)
    return r.json()


In [6]:
#Create and empty DF to store the results
cols = ['danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'type',
    'id',
    'uri',
    'track_href',
    'analysis_url',
    'duration_ms',
    'time_signature',]

song_features = pd.DataFrame(columns=cols)

In [7]:
#we can make a single request with a csv of up to 100 song ids at once. So we break the dataframe into smaller arrays to make multiple requests
splits = ceil(len(tracks)/100)
chunks = np.array_split(tracks, splits)

for i in range(len(chunks)):
    #transform the IDs from each chun into a csv list to be used at the api request
    song_ids = chunks[i]['sp_id'].to_string(header=False,index=False).replace('\n ',',').lstrip()   
    #make the api request
    r = get_song_features(song_ids)
    #normalize and append the reults to a final dataframe
    temp = pd.json_normalize(r['audio_features'])
    song_features = song_features.append(temp)

song_features.set_index('id', inplace=True)
song_features.to_csv('data/songs_features.csv')